In [1]:
#@title Install Dependencies
!pip install -q sentence-transformers datasets --upgrade

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 2.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.3/519.3 kB 16.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.5/7.5 MB 73.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 61.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 27.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 15.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 21.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 17.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 100.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 85.1 MB/s eta 0:00:00


In [2]:
#@title Download Data and Model

from sentence_transformers import SentenceTransformer
from transformers import AutoTokenizer
from datasets import load_dataset

import torch
from torch import nn
from torch.optim import Adam

device = 'cuda'

dataset = load_dataset("rotten_tomatoes")
dataset = dataset.with_format("torch")

model_ids = [
    "BAAI/bge-base-en",
    "intfloat/e5-base-v2",
    "thenlper/gte-base"
]
models = {}

for model_id in model_ids:
    model = SentenceTransformer(model_id).to(device)
    model.eval()
    models[model_id] = model

Generating train split:   0%|          | 0/8530 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1066 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1066 [00:00<?, ? examples/s]

In [3]:
#@title Compute Embeddings

from tqdm.auto import tqdm
import numpy as np

# embeddings and labels for each model
embeddings = {}
labels = {}

batch_size = 32
num_batches = len(dataset['train']) // batch_size

for model_id in model_ids:
  model = models[model_id]

  for i in tqdm(range(num_batches)):
    indices = [batch_size * i + j for j in range(batch_size)]
    text = [dataset['train'][i]['text'].lower() for i in indices]
    current_labels = [dataset['train'][i]['label'] for i in indices]

    # calculate embeddings
    current_embeddings = model.encode(text, normalize_embeddings=True)

    # add embeddings and labels
    if model_id not in embeddings:
      embeddings[model_id] = np.array(current_embeddings)
      labels[model_id] = np.array(current_labels)
    else:
      embeddings[model_id] = np.concatenate((embeddings[model_id], current_embeddings), axis=0)
      labels[model_id] = np.concatenate((labels[model_id], current_labels), axis=0)

  0%|          | 0/266 [00:00<?, ?it/s]

  0%|          | 0/266 [00:00<?, ?it/s]

  0%|          | 0/266 [00:00<?, ?it/s]

In [4]:
#@title Score Classification Accuracy

# score the classification accuracy of the different embeddings
# with a linear classifier using k-fold cross validation

from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

# number of folds
cross_validation_folds = 5

for model_name, model_embeddings in embeddings.items():
  model_labels = labels[model_name]

  # accuracy on each fold
  accuracies = []

  for fold in range(cross_validation_folds):
    # location of test set in dataset
    test_start = fold * len(model_embeddings) // cross_validation_folds
    test_end = (fold + 1) * len(model_embeddings) // cross_validation_folds

    # train set
    train_embeddings = np.concatenate((model_embeddings[:test_start], model_embeddings[test_end:]), axis=0)
    train_labels = np.concatenate((model_labels[:test_start], model_labels[test_end:]), axis=0)
    # test set
    test_embeddings = model_embeddings[test_start:test_end]
    test_labels = model_labels[test_start:test_end]

    # linear classifier
    classifier = Pipeline([
      ('scaler', StandardScaler()),
      ('classifier', LogisticRegression(max_iter=1000))
    ])
    classifier.fit(train_embeddings, train_labels)

    # score accuracy on test set
    predictions = classifier.predict(test_embeddings)
    accuracy = accuracy_score(test_labels, predictions)

    accuracies.append(accuracy)

  average_accuracy = np.mean(accuracies)
  print(f"{model_name}: {'%.3f' % average_accuracy}")


BAAI/bge-base-en: 0.801
intfloat/e5-base-v2: 0.824
thenlper/gte-base: 0.801
